# Load Spark Kernel
Using `findspark`, but many other ways exist too. See ([Installing Spark](https://spark.apache.org/downloads.html))

In [8]:
! echo $JAVA_HOME

/Library/Java/JavaVirtualMachines/jdk-14.0.1.jdk/Contents/Home


In [4]:
! echo $SPARK_HOME

/usr/local/spark


In [2]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="shparkley_example")

## Verify it's started using `jps`


In [7]:
! jps -lvm

11302 org.apache.spark.deploy.SparkSubmit pyspark-shell -Xmx1g
11415 jdk.jcmd/sun.tools.jps.Jps -lvm -Dapplication.home=/Library/Java/JavaVirtualMachines/jdk-14.0.1.jdk/Contents/Home -Xms8m -Djdk.module.main=jdk.jcmd


`pyspark-shell` is running -- looks good. 